In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import qp
from rail.plotting.pz_plotters import PZPlotterPointEstimateVsTrueHist2D

Change this to be the root of the current PZ working area

In [ ]:
pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'

In [ ]:
def calc_std(qp_dstn, grid):
    pdfs = qp_dstn.pdf(grid)
    norms = pdfs.sum(axis=1)
    means = np.sum(pdfs * grid, axis=1) / norms
    diffs = (np.expand_dims(grid, -1) - means).T
    wt_diffs = diffs * diffs * pdfs
    stds = np.sqrt((wt_diffs).sum(axis=1)/norms)
    return np.expand_dims(stds, -1)

In [ ]:
grid = np.linspace(0., 4., 401)

In [ ]:
algo = 'knn'
d = tables_io.read(f"{pz_dir}/data/test/dp1_matched_test.hdf5")
zmodes = []
stds = []
for algo in ['knn', 'cmnn', 'bpz', 'fzboost', 'lephare', 'dnf', 'tpz']:
    pz = qp.read(f"{pz_dir}/projects/dp1/data/gold_dp1_optimize/output_estimate_{algo}.hdf5")
    zmodes.append(np.squeeze(pz.ancil['zmode']))
    stds.append(np.squeeze(calc_std(pz, grid)))
redshifts = d['redshift']
zmodes = np.vstack(zmodes)
stds = np.vstack(stds)

In [ ]:
zmeta = np.mean(zmodes, axis=0)
zstd_1 = np.std(zmodes, axis=0)
zstd = np.mean(stds, axis=0)

In [ ]:
mask = zstd < 0.1
plotter = PZPlotterPointEstimateVsTrueHist2D()
out = plotter._make_2d_hist_plot('plot', redshifts, zmeta)

In [ ]:
def inv_var_weight(means, stds):
    weights = 1./(stds*stds)
    sum_weights = np.sum(weights, axis=0)
    zmean = np.sum(zmodes*weights, axis=0) / sum_weights    
    return zmean

In [ ]:
zmeans = inv_var_weight(zmodes, stds)

In [ ]:
mask = zstd_1 < 0.1
plotter = PZPlotterPointEstimateVsTrueHist2D()
out = plotter._make_2d_hist_plot('plot', redshifts, zmeans)

In [ ]:
mask = zstd_1 < 0.2
plotter = PZPlotterPointEstimateVsTrueHist2D()
out = plotter._make_2d_hist_plot('plot', redshifts[mask], zmeta[mask])

In [ ]:
mask.sum()/mask.size